In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Activation, Dense, Dropout, Flatten, BatchNormalization, Conv2D, MaxPooling2D, Lambda, Input, AveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing import image
import tensorflow as tf 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import random
import cv2
import os

In [11]:
# import csv to match correct condition
train= pd.read_csv("https://raw.githubusercontent.com/Coachnmomof3/UCB_COVID_Prediction_Model/james_demott/connect_core_data_set_csv.csv", dtype=str)
test = pd.read_csv("https://raw.githubusercontent.com/Coachnmomof3/UCB_COVID_Prediction_Model/james_demott/connect_core_data_set_csv.csv")

# Define Train/Test Folder path
train_folder = "./Resources/data_upload_v2/images"
test_folder = "./Resources/data_upload_v2/images"


In [12]:
train.head()

,condition,file,test_train
0,Atelectasis,patient00024-study2-view1_frontal.jpg,train
1,Atelectasis,patient00109-study1-view1_frontal.jpg,train
2,Atelectasis,patient00150-study2-view1_frontal.jpg,train
3,Atelectasis,patient00224-study1-view1_frontal.jpg,train
4,Atelectasis,patient00262-study2-view1_frontal.jpg,train


In [19]:
train.condition.value_counts()

No_Finding                    3400
Edema                          240
Fracture                       238
Lung_Opacity                   238
Atelectasis                    234
Enlarged_Cardiomediastinum     230
Support_Devices                226
Pneumothorax                   226
Consolidation                  224
Cardiomegaly                   224
Lung_Lesion                    218
Pneumonia                      194
Covid                          184
Pleural_Other                  108
Name: condition, dtype: int64

In [22]:
# Directory with Atelectasis
atele_dir=os.path.join("./Resources/data_upload_v2/images/Atelectasis")
# Check filenames 
image_names=list(os.listdir("./Resources/data_upload_v2/images"))[1:]
image_names.sort()

df =pd.DataFrame.from_records({"file_name":os.listdir(atele_dir),"condition": "Atelectasis"})

for f in image_names[1:]:
    folder_path="./Resources/data_upload_v2/images/" + f
    temp_df= pd.DataFrame.from_records({"file_name":os.listdir(folder_path),"condition": f })
    df=df.append(temp_df)
    
df["condition"].value_counts()

No_Finding                    1898
Covid                          185
Pneumothorax                   164
Lung_Opacity                   156
Edema                          155
Support_Devices                154
Consolidation                  145
Enlarged_Cardiomediastinum     143
Atelectasis                    143
Fracture                       142
Cardiomegaly                   136
Lung_Lesion                    127
Pneumonia                      118
Pleural_Other                   99
Name: condition, dtype: int64

In [23]:
df.head()

,condition,file_name
0,Atelectasis,patient02592-study2-view1_frontal.jpg
1,Atelectasis,patient01340-study2-view1_frontal.jpg
2,Atelectasis,patient00109-study1-view1_frontal.jpg
3,Atelectasis,patient01192-study1-view1_frontal.jpg
4,Atelectasis,patient01608-study1-view1_frontal.jpg


In [13]:
# Define image generator 
train_gen= ImageDataGenerator(rotation_range=45, rescale=1/255, horizontal_flip=True)

test_gen=ImageDataGenerator(rescale=1/255)


In [24]:
train_data = train_gen.flow_from_dataframe(dataframe=df, 
    directory = train_folder, 
    x_col = "file_name", 
    y_col = "condition", seed = 30,
    batch_size = 128, shuffle = True, 
    class_mode="categorical",
    subset="training",                                      
    classes=["Covid", "Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Enlarged_Cardiomediastinum", "Fracture", 
    "Lung_Lesion", "Lung_Opacity", "No_Finding", "Pleural_Other", "Pneumonia", "Pneumothorax", "Support_Devices"],
    target_size = (200,200))

# test_data = test_gen.flow_from_dataframe(dataframe = test, 
# directory = test_folder, x_col = "file", 
# y_col = None,
# batch_size = 128, shuffle = False, 
# class_mode=None,target_size = (200,200))

Found 0 validated image filenames belonging to 14 classes.


/Users/bijansamimi/anaconda3/envs/mlenv/lib/python3.7/site-packages/keras_preprocessing/image/dataframe_iterator.py:282: UserWarning: Found 3765 invalid image filename(s) in x_col="file_name". These filename(s) will be ignored.
  .format(n_invalid, x_col)
